In [10]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import json

In [11]:
parks_geoparquet = "data" + os.sep + "parks.geoparquet"
zones_geoparquet = "data" + os.sep + "zones.geoparquet"

In [12]:
parks = gpd.read_parquet(parks_geoparquet)
zones = gpd.read_parquet(zones_geoparquet)

In [13]:
parks.currentTimestamp.max()

Timestamp('2025-03-11 13:21:00')